In [ ]:
import requests
from flask import Flask, request, redirect
import threading

In [4]:
CLIENT_ID = "127031"
CLIENT_SECRET = "232ecc1f360f883b868b29f801dc2f1bab7b3078"

In [9]:
app = Flask(__name__)
authorization_code = None

@app.route('/exchange_token')
def exchange_token():
    global authorization_code
    authorization_code = request.args.get('code')
    return "Authorization code received. You can close this window."

def run_flask_app():
    app.run(port=5000)

def get_authorization_code(client_id):
    # Construct the authorization URL
    authorization_url = (
        f"http://www.strava.com/oauth/authorize?client_id={client_id}"
        "&response_type=code&redirect_uri=http://localhost:5000/exchange_token"
        "&approval_prompt=force&scope=read_all,profile:read_all,activity:read_all,activity:write"
    )
    # Open the authorization URL in the default web browser
    import webbrowser
    webbrowser.open(authorization_url)

if __name__ == '__main__':
    client_id = CLIENT_ID  # Replace with your client ID

    # Start Flask app in a separate thread
    threading.Thread(target=run_flask_app).start()

    # Open the authorization URL
    get_authorization_code(client_id)

    # Wait until the authorization code is received
    while authorization_code is None:
        pass  # Busy wait, could be improved with threading conditions or similar

    print(f"Authorization code: {authorization_code}")

    # Exchange the authorization code for an access token
    client_secret = CLIENT_SECRET  # Replace with your client secret
    token_url = 'https://www.strava.com/oauth/token'
    data = {
        'client_id': client_id,
        'client_secret': client_secret,
        'code': authorization_code,
        'grant_type': 'authorization_code'
    }
    response = requests.post(token_url, data=data)
    token_info = response.json()

    print(f"Access token: {token_info['access_token']}")
    print(f"Refresh token: {token_info['refresh_token']}")


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Authorization code: ece590cf36932f92a76388e5289f44307c1496a9
Access token: 57f220c2f577ef5f578d198b74ff9cffd6f05385
Refresh token: ce5a15a2f58734fd99404df19442fb4688c538be


In [12]:
access_token = token_info['access_token']

In [49]:
activities_url = 'https://www.strava.com/api/v3/athlete/activities'
headers = {'Authorization': f'Bearer {access_token}'}
    
response = requests.get(activities_url, headers=headers)

In [50]:
response.json()

[{'resource_state': 2,
  'athlete': {'id': 86360204, 'resource_state': 1},
  'name': '🌊 Seuil - 4*3mn @5’55',
  'distance': 6223.8,
  'moving_time': 2594,
  'elapsed_time': 2737,
  'total_elevation_gain': 12.0,
  'type': 'Run',
  'sport_type': 'Run',
  'workout_type': 0,
  'id': 11511427976,
  'start_date': '2024-05-27T17:12:11Z',
  'start_date_local': '2024-05-27T19:12:11Z',
  'timezone': '(GMT+01:00) Europe/Paris',
  'utc_offset': 7200.0,
  'location_city': None,
  'location_state': None,
  'location_country': None,
  'achievement_count': 5,
  'kudos_count': 22,
  'comment_count': 0,
  'athlete_count': 2,
  'photo_count': 0,
  'map': {'id': 'a11511427976',
   'summary_polyline': 'ym_iHufuLh@kAT{@DYP]Fm@Le@X}ATcARo@Ds@`@gALo@Ni@TyATm@LIJBLEP]Li@ZKLI`Ac@`A]ZQHARMZM^WHAJBZCRIHM`@YNGf@YF?XSNGf@Ir@[H@JNVFNRZf@Xr@?TAn@B\\Rx@bAbGBXJ^?JMZAN^fBp@zFB~@Jt@HnAFrB@`BGh@@fBLzBAh@@VAxCGlAO~ACj@eAtGE^u@vDw@rFMj@W`@EP?JBT_@lCYlA_A|CKVETGRA\\Op@o@pBm@zAQr@O|@qBrCyAdB?C`C}C|@}AN_@Jm@v@uBbAaDJg@nAuDpAuE

In [26]:
activity = response.json()[0]
activity

{'resource_state': 2,
 'athlete': {'id': 86360204, 'resource_state': 1},
 'name': '🌊 Seuil - 4*3mn @5’55',
 'distance': 6223.8,
 'moving_time': 2594,
 'elapsed_time': 2737,
 'total_elevation_gain': 12.0,
 'type': 'Run',
 'sport_type': 'Run',
 'workout_type': 0,
 'id': 11511427976,
 'start_date': '2024-05-27T17:12:11Z',
 'start_date_local': '2024-05-27T19:12:11Z',
 'timezone': '(GMT+01:00) Europe/Paris',
 'utc_offset': 7200.0,
 'location_city': None,
 'location_state': None,
 'location_country': None,
 'achievement_count': 5,
 'kudos_count': 22,
 'comment_count': 0,
 'athlete_count': 2,
 'photo_count': 0,
 'map': {'id': 'a11511427976',
  'summary_polyline': 'ym_iHufuLh@kAT{@DYP]Fm@Le@X}ATcARo@Ds@`@gALo@Ni@TyATm@LIJBLEP]Li@ZKLI`Ac@`A]ZQHARMZM^WHAJBZCRIHM`@YNGf@YF?XSNGf@Ir@[H@JNVFNRZf@Xr@?TAn@B\\Rx@bAbGBXJ^?JMZAN^fBp@zFB~@Jt@HnAFrB@`BGh@@fBLzBAh@@VAxCGlAO~ACj@eAtGE^u@vDw@rFMj@W`@EP?JBT_@lCYlA_A|CKVETGRA\\Op@o@pBm@zAQr@O|@qBrCyAdB?C`C}C|@}AN_@Jm@v@uBbAaDJg@nAuDpAuEZ{@F_@Bq@^kAb@{B@k@Hy@Je@

In [37]:
from geopy.geocoders import Nominatim

def get_city_from_coordinates(latitude, longitude):
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    return city

# Exemples de coordonnées (latitude, longitude)
latitude = 48.826939864084125
longitude = 2.241187132894993

city = get_city_from_coordinates(latitude, longitude)
print(f"City: {city}")
##############

# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="my-app")
# location = geolocator.reverse(f"{latitude}, {longitude}")
# print(location.address)


City: Boulogne-Billancourt


In [44]:
start_city, end_city = get_city_from_coordinates(activity['start_latlng'][0], activity['start_latlng'][1]), get_city_from_coordinates(activity['end_latlng'][0], activity['end_latlng'][1])

texte = f"""
J'ai fait la séance de {activity['type']} suivante : {activity['name']}, le {activity['start_date']}.\n
Je suis parti de {start_city} et ai fini à {end_city}.\n
La séance a duré en tout {activity['elapsed_time']} secondes, dont {activity['moving_time']} d'activité.\n
La distance parcourue est de {activity['distance']}m, avec un dénivelé positif de {activity['total_elevation_gain']}.\n
J'ai couru en moyenne à {activity['average_speed']} m/s, avec une pointe à {activity['max_speed']}.\n
Mon rythme cardiaque était en moyenne de {activity['average_heartrate']}, avec une pointe à {activity['max_heartrate']}.\n
"""
print(texte)


J'ai fait la séance de Run suivante : 🌊 Seuil - 4*3mn @5’55, le 2024-05-27T17:12:11Z.

Je suis parti de Boulogne-Billancourt et ai fini à Boulogne-Billancourt.

La séance a duré en tout 2737 secondes, dont 2594 d'activité.

La distance parcourue est de 6223.8m, avec un dénivelé positif de 12.0.

J'ai couru en moyenne à 2.399 m/s, avec une pointe à 4.5.

Mon rythme cardiaque était en moyenne de 143.5, avec une pointe à 171.0.




In [ ]:
context_prompt = """
Tu es un agent spécialisé dans le résumé humoristique/sarcastique de séances de sport, en te basant sur un texte récapitulatif de la séance, 
avec des informations comme la vitesse, le rythme cardique, la distance, les villes de début et de fin, le titre de la séance.
Il te faut être drôle, ce texte sera publié en description de chaque post Strava, et donc visible auprès de mes abonnés.

Voici le texte récapitulatif :


"""

In [82]:
id = 11099445685
activity_lap_url = f'https://www.strava.com/api/v3/activities/{id}/laps'
headers = {'Authorization': f'Bearer {access_token}'}
    
response = requests.get(activity_lap_url, headers=headers)

In [83]:
def mps_to_minspeed(speed_mps):
    # Conversion de m/s en km/min
    speed_kmpm = speed_mps * 60 / 1000  # Convertir m/s en km/min
    if speed_kmpm != 0:
        time_per_km = 1 / speed_kmpm  # Temps par kilomètre en minutes
        minutes = int(time_per_km)  # Partie entière représente les minutes
        seconds = (time_per_km - minutes) * 60  # Convertir la partie décimale en secondes
    else:
        minutes = float('inf')
        seconds = float('inf')
    return str(minutes) + ',' + str(round(seconds))

In [84]:
for lap in response.json():
    t = lap['elapsed_time'] // 60
    d = round(lap['distance'])
    s = mps_to_minspeed(lap['average_speed'])
    h = round(lap['average_heartrate'])
    print(f'Distance : {d} mètres / Temps : {t} minutes / Vitesse : {s} mn/km / Heartrate : {h} bpm')

Distance : 731 mètres / Temps : 5 minutes / Vitesse : 6,50 mn/km / Heartrate : 139 bpm
Distance : 1000 mètres / Temps : 7 minutes / Vitesse : 6,31 mn/km / Heartrate : 147 bpm
Distance : 1000 mètres / Temps : 7 minutes / Vitesse : 6,38 mn/km / Heartrate : 154 bpm
Distance : 1000 mètres / Temps : 7 minutes / Vitesse : 6,42 mn/km / Heartrate : 156 bpm
Distance : 1000 mètres / Temps : 9 minutes / Vitesse : 6,47 mn/km / Heartrate : 151 bpm
Distance : 1000 mètres / Temps : 6 minutes / Vitesse : 6,57 mn/km / Heartrate : 162 bpm
Distance : 1000 mètres / Temps : 11 minutes / Vitesse : 7,4 mn/km / Heartrate : 161 bpm
Distance : 1000 mètres / Temps : 6 minutes / Vitesse : 6,53 mn/km / Heartrate : 157 bpm
Distance : 1000 mètres / Temps : 7 minutes / Vitesse : 6,43 mn/km / Heartrate : 158 bpm
Distance : 1000 mètres / Temps : 6 minutes / Vitesse : 6,48 mn/km / Heartrate : 160 bpm
Distance : 788 mètres / Temps : 5 minutes / Vitesse : 6,35 mn/km / Heartrate : 163 bpm
